# Geos API Example

## Run this example in [Colab](https://colab.research.google.com/github/SignalOceanSdk/SignalSDK/blob/master/docs/examples/jupyter/GeoAPI/GeoAPI%20Use%20Case.ipynb) 

## Setup

Install the Signal Ocean SDK:

In [ ]:
pip install signal-ocean

Set your subscription key acquired here: https://apis.signalocean.com/profile

In [79]:
signal_ocean_api_key = '6de2b1078a424f5cb70f571948fc4ba1' #replace with your subscription key

In [80]:
from signal_ocean import Connection
from signal_ocean.geos import GeosAPI
import numpy as np
import pandas as pd
import requests
import json

In [81]:
connection = Connection(signal_ocean_api_key)
api = GeosAPI(connection)

## Scope

In this notebook we will show how you can get all the different kinds of Geo data available.

There are three kinds of Geo data:
* Areas
* Countries
* Ports
* GeoAssets

## Areas

### Get all available areas

In [82]:
all_areas = api.get_areas()

In [83]:
df_areas = pd.DataFrame([a.__dict__ for a in all_areas])

In [84]:
df_areas.head(10)

,area_id,location_taxonomy_id,location_taxonomy_name,area_name,parent_area_id,parent_area_name
0,-32,7,Level3,Not set,NaN,None
1,-31,7,Level3,Unknown,NaN,None
2,-22,6,Level2,Not set,-32.0,Not set
3,-21,6,Level2,Unknown,-31.0,Unknown
4,-12,5,Level1,Not set,-22.0,Not set
5,-11,5,Level1,Unknown,-21.0,Unknown
6,-2,4,Level0,Not set,-12.0,Not set
7,-1,4,Level0,Unknown,-11.0,Unknown
8,2,5,Level1,Arabian Gulf,89.0,Arabian Gulf
9,3,5,Level1,Arctic Ocean & Barents Sea,25021.0,Baltic / North Sea


This can create a tree-like structure.

**Example** 
* West
    * Black Sea / Sea Of Marmara
    * Atlantic America
    * Baltic / North Sea
    * Mediterranean / UK Continent 
        * UK Continent
        * Mediterranean
            * West Mediterranean
            * East Mediterranean
            * Central Mediterranean

#### Get a dataframe with all area level names

We can get a dataframe with the complete area tree by gradually merging an area type with the one below it.

In [85]:
df_areas_all = df_areas[df_areas['location_taxonomy_id'] == 7].merge(df_areas,
                                                             how = 'left',
                                                             left_on = 'area_id',
                                                             right_on = 'parent_area_id',
                                                             suffixes = ['_type7','_type6'])[['area_id_type7',
                                                                                              'area_name_type7',
                                                                                              'area_id_type6',
                                                                                              'area_name_type6']]\
                                                      .merge(df_areas,
                                                             how = 'left',
                                                             left_on = 'area_id_type6',
                                                             right_on = 'parent_area_id')[['area_id_type7',
                                                                                           'area_name_type7',
                                                                                           'area_id_type6',
                                                                                           'area_name_type6',
                                                                                           'area_id',
                                                                                           'area_name']]\
                                                      .merge(df_areas,
                                                             how = 'left',
                                                             left_on = 'area_id',
                                                             right_on = 'parent_area_id',
                                                             suffixes = ['_type5','_type4'])[['area_id_type7',
                                                                                              'area_name_type7',
                                                                                              'area_id_type6',
                                                                                              'area_name_type6',
                                                                                              'area_id_type5',
                                                                                              'area_name_type5',
                                                                                              'area_id_type4',
                                                                                              'area_name_type4']]


In [86]:
df_areas_all.head(10)

,area_id_type7,area_name_type7,area_id_type6,area_name_type6,area_id_type5,area_name_type5,area_id_type4,area_name_type4
0,-32,Not set,-22,Not set,-12,Not set,-2,Not set
1,-31,Unknown,-21,Unknown,-11,Unknown,-1,Unknown
2,84,East,89,Arabian Gulf,2,Arabian Gulf,24777,Arabian Gulf
3,84,East,99,Far East,17,China / Taiwan,24666,North China
4,84,East,99,Far East,17,China / Taiwan,24725,South China
5,84,East,99,Far East,17,China / Taiwan,24726,Central China
6,84,East,99,Far East,17,China / Taiwan,24729,Taiwan
7,84,East,99,Far East,19,Korea / Japan,24715,Korea
8,84,East,99,Far East,19,Korea / Japan,24730,Japan Island
9,84,East,99,Far East,20,Pacific Islands,24782,Pacific Islands


From there we can recreate the area tree example from above, by filetring a specific type 3 area.

In [87]:
df_areas_all[df_areas_all['area_name_type7'] == 'West']

,area_id_type7,area_name_type7,area_id_type6,area_name_type6,area_id_type5,area_name_type5,area_id_type4,area_name_type4
34,25028,West,93,Black Sea / Sea Of Marmara,7,Black Sea / Sea Of Marmara,24583,Black Sea
35,25028,West,93,Black Sea / Sea Of Marmara,7,Black Sea / Sea Of Marmara,24584,Sea of Marmara
36,25028,West,25019,Atlantic America,9,Caribs,24746,Caribs
37,25028,West,25019,Atlantic America,10,East Coast Central America,24600,East Coast Central America
38,25028,West,25019,Atlantic America,12,East Coast Canada,24732,Great Lakes
39,25028,West,25019,Atlantic America,12,East Coast Canada,24740,Canada Atlantic Coast
40,25028,West,25019,Atlantic America,13,East Coast Mexico,24671,East Coast Mexico
41,25028,West,25019,Atlantic America,15,East Coast South America,24767,Brazil
42,25028,West,25019,Atlantic America,15,East Coast South America,24769,Argentina & Uruguay
43,25028,West,25019,Atlantic America,32,US Atlantic Coast,24747,US Atlantic Coast


## Countries

### Get available countries

In [88]:
all_countries = api.get_countries()

In [89]:
df_countries = pd.DataFrame([a.__dict__ for a in all_countries])

In [90]:
df_countries.head(10)

,country_id,country_name,location_taxonomy_id,location_taxonomy_name
0,-104,Korea,3,Country
1,-103,Arabian Gulf,3,Country
2,-102,Caribs,3,Country
3,-101,East Mediterranean,3,Country
4,-2,Not set,3,Country
5,-1,Unknown,3,Country
6,3,Azores,3,Country
7,4,Czechoslovakia,3,Country
8,5,Madeira,3,Country
9,6,Neutral Zone (between Saudi Arabia & Iraq),3,Country


## Ports

### Get availabe ports

In [91]:
all_ports = api.get_ports()

In [92]:
df_ports = pd.DataFrame([a.__dict__ for a in all_ports])

In [93]:
df_ports.head(10)

,port_id,location_taxonomy_id,location_taxonomy_name,port_name,unlocode,country_id,country_name,area_id_level0,area_name_level0,area_id_level1,area_name_level1,area_id_level2,area_name_level2,area_id_level3,area_name_level3,latitude,longitude
0,-104,2,Port,Uncertain,None,-104,Korea,24715,Korea,19,Korea / Japan,99,Far East,84,East,0.000000,0.000000
1,-103,2,Port,Uncertain,None,-103,Arabian Gulf,24777,Arabian Gulf,2,Arabian Gulf,89,Arabian Gulf,84,East,0.000000,0.000000
2,-102,2,Port,Uncertain,None,-102,Caribs,24746,Caribs,9,Caribs,25019,Atlantic America,25028,West,0.000000,0.000000
3,-101,2,Port,Uncertain,None,-101,East Mediterranean,24737,East Mediterranean,25013,Mediterranean,25025,Mediterranean / UK Continent,25028,West,0.000000,0.000000
4,-2,2,Port,Not Set,None,-2,Not set,-1,Unknown,-11,Unknown,-21,Unknown,-31,Unknown,0.000000,0.000000
5,-1,2,Port,Unknown,None,-1,Unknown,-1,Unknown,-11,Unknown,-21,Unknown,-31,Unknown,0.000000,0.000000
6,3153,2,Port,Fujairah,AE FJR,9,United Arab Emirates,24777,Arabian Gulf,2,Arabian Gulf,89,Arabian Gulf,84,East,25.197490,56.383900
7,3154,2,Port,Das Island,AE DAS,9,United Arab Emirates,24777,Arabian Gulf,2,Arabian Gulf,89,Arabian Gulf,84,East,25.133394,52.919978
8,3155,2,Port,Fateh Terminal,AE FAT,9,United Arab Emirates,24777,Arabian Gulf,2,Arabian Gulf,89,Arabian Gulf,84,East,25.577130,54.468500
9,3156,2,Port,Mina Saqr,AE MSA,9,United Arab Emirates,24777,Arabian Gulf,2,Arabian Gulf,89,Arabian Gulf,84,East,25.974710,55.940320


## Using all of the data above

### Get all ports in a specific level 0 area

In [94]:
area_name = 'US Gulf'

In [95]:
df_Ports_In_Area = df_areas[  (df_areas['area_name'] == area_name ) 
                            & (df_areas['location_taxonomy_id'] == 4)].merge(df_ports,
                                                                     how = 'left',
                                                                     left_on = 'area_id',
                                                                     right_on = 'area_id_level0',
                                                                     suffixes = ['_area','_port'])[['port_id','port_name','area_id','area_name']]

In [96]:
df_Ports_In_Area.head(10)

,port_id,port_name,area_id,area_name
0,3838,Baton Rouge,24676,US Gulf
1,3841,Brownsville,24676,US Gulf
2,3845,Corpus Christi,24676,US Gulf
3,3849,Freeport (Texas),24676,US Gulf
4,3850,Galveston,24676,US Gulf
5,3851,Garyville,24676,US Gulf
6,3853,Houston,24676,US Gulf
7,3856,Lake Charles,24676,US Gulf
8,3858,Loop,24676,US Gulf
9,3861,Mobile,24676,US Gulf


### Get all ports in a specific country

In [97]:
country_name = 'United Arab Emirates'

In [98]:
df_Ports_In_Country = df_countries[df_countries['country_name'] == country_name].merge(df_ports,
                                                                               how = 'left',
                                                                               left_on = 'country_id',
                                                                               right_on = 'country_id',
                                                                               suffixes = ['_country','_port'])[['port_id','port_name','country_id','country_name_country']]

In [99]:
df_Ports_In_Country.head(10)

,port_id,port_name,country_id,country_name_country
0,3153,Fujairah,9,United Arab Emirates
1,3154,Das Island,9,United Arab Emirates
2,3155,Fateh Terminal,9,United Arab Emirates
3,3156,Mina Saqr,9,United Arab Emirates
4,3157,Jebel Ali,9,United Arab Emirates
5,3158,Jebel Dhanna,9,United Arab Emirates
6,3159,Mubarras Island,9,United Arab Emirates
7,3160,Ruwais,9,United Arab Emirates
8,3161,Sharjah,9,United Arab Emirates
9,3162,Zirku Island,9,United Arab Emirates


### Get all areas in a country

Some countries can span across multiple type 0 areas. We can make that connection through **ports**.

In [100]:
country_name = 'United States'

In [101]:
df_countries[df_countries['country_name'] == country_name].merge(df_ports,
                                                         how = 'left',
                                                         left_on = 'country_id',
                                                         right_on = 'country_id',
                                                         suffixes = ['_country',None])[['port_id','port_name','country_id','country_name_country','area_id_level0']]\
                                                  .merge(df_areas,
                                                         how = 'left',
                                                         left_on = 'area_id_level0',
                                                         right_on = 'area_id',
                                                         suffixes = (None, '_area'))[['country_id','country_name_country','area_id', 'area_name']].area_name.unique()

array(['US Atlantic Coast', 'US North Pacific', 'Alaska', 'US Gulf',
       'US West Coast', 'Pacific Islands', 'US Mainland', 'Great Lakes'],
      dtype=object)

#### Find the top 10 Countries with the most Ports

In [102]:
df_Ports_country_count =  df_ports.merge(df_countries,
                                         how = 'left',
                                         left_on = 'country_id',
                                         right_on = 'country_id',
                                         suffixes =['_port','_country'])[['port_id','port_name','country_id','country_name_country']] 

In [103]:
df_Ports_country_count.country_name_country.value_counts().head(10)

United States         1025
China                  861
Japan                  643
Indonesia              449
Brazil                 300
Australia              297
Canada                 278
United Kingdom         269
Russian Federation     236
India                  233
Name: country_name_country, dtype: int64